# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 852.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.0 MB/s eta 0:00:00


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [5]:
# Data loading functions. Uncomment the one you want to use
#from worcgist.load_data import load_data
#from worclipo.load_data import load_data
from worcliver.load_data import load_data
#from ecg.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



The number of samples: 186
The number of columns: 494


## Pre-processing
Feature Cleaning:

1. Loading the full dataset from Excel.
2. Identifying and removing features with excessive outliers using the IQR method.
3. Winsorizing (capping) extreme values in the remaining features to reduce their influence.
4. Printing the new dataset dimensions and how many features were removed.

In [81]:
# Imports
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from pathlib import Path

current_dir = Path.cwd()
dir_results = current_dir / 'Results'

# Load the dataset from an Excel file into a DataFrame
df = pd.read_excel('FullDataset.xlsx')
print(f"Original shape: {df.shape}")  # Shows initial number of rows and columns (samples, features)

# Keep only the numeric columns (e.g., float, int) for outlier analysis
numeric_df = df.select_dtypes(include='number')

# Dictionary to store number of outliers found in each column
outlier_counts = {}

# Loop over all numeric features to detect outliers
for col in numeric_df.columns:
    Q1 = numeric_df[col].quantile(0.25)     # First quartile (25th percentile)
    Q3 = numeric_df[col].quantile(0.75)     # Third quartile (75th percentile)
    IQR = Q3 - Q1                           # Interquartile range
    lower_bound = Q1 - 5 * IQR              # Conservative lower outlier threshold
    upper_bound = Q3 + 5 * IQR              # Conservative upper outlier threshold

    # Identify outliers and count them
    outliers = numeric_df[(numeric_df[col] < lower_bound) | (numeric_df[col] > upper_bound)]
    outlier_counts[col] = len(outliers)

# Create a DataFrame from the dictionary to make it easy to sort/view
outlier_df = pd.DataFrame.from_dict(outlier_counts, orient='index', columns=['outlier_count'])
outlier_df = outlier_df.sort_values(by='outlier_count', ascending=False)

# Define a threshold: if a feature has more than 17 outliers, we consider it too noisy
threshold = 17
features_to_drop = outlier_df[outlier_df['outlier_count'] > threshold].index.tolist()

# Drop those features from the original DataFrame
df = df.drop(columns=features_to_drop)

# Identify the features that are still allowed (with 17 or fewer outliers)
features_to_winsorize = outlier_df[outlier_df['outlier_count'] <= threshold].index.tolist()

# Apply outlier capping to these "acceptable" features
for feature in features_to_winsorize:
    if feature in df.columns:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 5 * IQR
        upper_bound = Q3 + 5 * IQR

        # Clip values that fall outside the acceptable range
        df[feature] = df[feature].clip(lower=lower_bound, upper=upper_bound)

# Print new shape of the cleaned dataset



print(f"Shape after cleaning: {df.shape}")
print(f"Removed {len(features_to_drop)} features with more than {threshold} outliers.")

numeric_df = df.select_dtypes(include='number').drop(columns=['label'])
X_clean = numeric_df.values

print(f"Shape before removing constant features: {X_clean.shape}")

# Verwijder constant features (met variantie == 0)
selector = VarianceThreshold(threshold=0.0)
X_clean = selector.fit_transform(X_clean)
y_clean = df['label'].values

print(f"Shape after removing constant features: {X_clean.shape}")



Original shape: (186, 494)
Shape after cleaning: (186, 487)
Removed 7 features with more than 17 outliers.
Shape before removing constant features: (186, 486)
Shape after removing constant features: (186, 452)


## Feature selection

This function performs nested cross-validation for robust model evaluation and hyperparameter tuning using:
- Outer loop: Evaluates generalization performance
- Inner loop: Tunes hyperparameters using GridSearchCV
- RFECV: Performs recursive feature elimination within each outer fold

It returns performance scores, selected hyperparameters, and feature selection masks across multiple repeated trials

In [91]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_classif

# difficult nested score to see best params. Perform grid-search with nested cross-validation for hyperparameter tuning
def safe_f_classif(X, y):
    """Wrapper around sklearn's _f_classif to handle divide-by-zero."""
    with np.errstate(divide='ignore', invalid='ignore'):
        return f_classif(X, y)
    
def nested_cross_validation(NUM_TRIALS : int, clf_class, param_grid : dict, X : np.ndarray, y : np.ndarray, mask = None) -> tuple:
    """
    Perform nested cross-validation for hyperparameter tuning.
    This function uses RFECV for feature selection and SelectKBest for feature selection.
    It evaluates the model using accuracy as the scoring metric.
    
    Parameters:
    - NUM_TRIALS: Number of trials to run
    - clf_class: The machine learning classifier to evaluate
    - param_grid: Hyperparameter grid for GridSearchCV
    - X: Feature matrix
    - y: Target vector
    
    Returns:
    - nested_scores: Array of scores from each trial
    - df_best_params: DataFrame of best parameters for each trial
    - df_masks: DataFrame of all feature selection masks
    """
    
    nested_scores = np.zeros(NUM_TRIALS)      # To store average scores per trial
    all_best_params = []                      # To store best hyperparameters per trial
    all_feature_masks = []                    # To store feature selection masks per trial

    for i in range(NUM_TRIALS):
        # Outer and inner stratified cross-validation splitters
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
        RFECV_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i+1000)
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i+2000)

        # Store the best parameters and scores for each outer fold
        best_params = []     # For storing best parameters for each outer fold
        outer_scores = []    # AUC scores per outer fold
        trial_masks = []     # Feature selection masks per outer fold

        k = 0
        for train_idx, test_idx in outer_cv.split(X, y):
            k += 1
            print(f"Outer fold {k}")
            # Split data into training and test for this outer fold
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            # Feature selection via SelectKBest using the outer CV folds
            # SelectKBest is used to select the top k features based on ANOVA F-value
            # You can delete this if you want to use all features, computation time will be longer!
            # selector_kbest = SelectKBest(score_func=safe_f_classif, k=100)              
            
            # Fit the selector on the training data, not the entire dataset
            # This can be deteled, but it will take longer to compute!
            # Fit SelectKBest on outer training set
            # and transform both the training and test sets
            # selector_kbest.fit(X_train, y_train)  # Fit SelectKBest on outer training set
            # X_train = selector_kbest.transform(X_train)  # Transform outer training set 
            # X_test = selector_kbest.transform(X_test)  # Transform outer test set        
            
            if mask is None:
            # Feature selection via RFECV using the inner CV folds
                selector_RFECV = RFECV(
                    estimator=RandomForestClassifier(),
                    step=1,
                    cv=RFECV_cv,
                    scoring='accuracy'
                    )


                selector_RFECV.fit(X_train, y_train)  # Fit RFECV on outer training set  
                trial_masks.append(selector_RFECV.support_)  # Save which features were kept
                print("RFECV done")
                # Reduce training and test sets to selected features
                X_train = selector_RFECV.transform(X_train)
                X_test = selector_RFECV.transform(X_test)
            else:
                # Use the provided mask to select features
                
                mask_used = mask[f'outer_{k}'].values[i-1]

                X_train = X_train[:, mask_used]
                X_test = X_test[:, mask_used]
                trial_masks.append(mask_used)
            # Build a fresh pipeline for each fold
            pipeline = Pipeline([
                ("scaler", MinMaxScaler()),       # Normalize features to [0, 1]
                ("clf_class", clf_class)          # Insert model
            ])
                        
            # Create a new GridSearchCV for each inner fold
            grid_search = GridSearchCV(pipeline, param_grid, cv=inner_cv, scoring="accuracy", n_jobs=-1)
            
            # Fit GridSearchCV on the training data
            grid_search.fit(X_train, y_train)
            
            # Get the best model from inner CV
            best_model = grid_search.best_estimator_
            
            # Evaluate the best model on the outer test fold
            y_pred = best_model.predict(X_test)
            outer_score = accuracy_score(y_test, y_pred)
            # Append the best parameters and outer score
            best_params.append(grid_search.best_params_)
            outer_scores.append(outer_score)
        
        # Store the mean score for this trial
        nested_scores[i] = np.mean(outer_scores)
        all_best_params.append([i, best_params])
        all_feature_masks.append([i, trial_masks])

    # Format best params and feature masks into DataFrames
    df_best_params = pd.DataFrame([best_params for _, best_params in all_best_params], columns=["outer_1", "outer_2", "outer_3", "outer_4", "outer_5"])
    df_masks = pd.DataFrame([trial_masks for _, trial_masks in all_feature_masks], columns=["outer_1", "outer_2", "outer_3", "outer_4", "outer_5"])


    # Print results
    print(f"Average performance across {NUM_TRIALS} trials: {np.mean(nested_scores):.4f} ± {np.std(nested_scores):.4f}")

    return nested_scores, df_best_params, df_masks


# Code for the KNN classifier
This code is used to perform nested cross-validation for hyperparameter tuning of a K-Nearest Neighbors (KNN) classifier.
It includes feature selection using SelectKBest and RFECV, and it evaluates the model's performance using accuracy as the scoring metric.

In [ ]:
# Imports
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'clf_class__n_neighbors': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
}

clf_class = KNeighborsClassifier()

# NUM_TRIALS = 1 -> could be changed when stronger computer available
nested_scores_knn, df_best_params_knn, df_masks_knn = nested_cross_validation(1,clf_class, param_grid, X_clean, y_clean)

# Save the results to CSV files
df_best_params_knn.to_csv(dir_results / "best_params_knn.csv", index=False)
df_masks_knn.to_csv(dir_results / 'masks_knn.csv', index=False)

for i in df_masks_knn:
    print(f"{i}: {df_masks_knn[i].values[0].sum()} features selected")

best_params_knn_ravel= pd.Series(df_best_params_knn.values.ravel())
best_params_knn_ravel.value_counts()[0:10]

Outer fold 1


# Code for the Naive Bayes
This code is used to perform nested cross-validation for hyperparameter tuning of a Naive Bayes classifier.
It includes feature selection using SelectKBest and RFECV, and it evaluates the model's performance using accuracy as the scoring metric.

In [87]:
# Imports
from sklearn.naive_bayes import GaussianNB

param_grid = {
    'clf_class__var_smoothing': [1*10**-12, 1*10**-11, 1*10**-10, 1*10**-9, 1*10**-8, 1*10**-7, 1*10**-6, 1*10**-5, 1*10**-4, 1*10**-3, 1*10**-2, 1*10**-1],
}

clf_class = GaussianNB()

# NUM_TRIALS = 1 -> could be changed when stronger computer available
nested_scores_naive, df_best_params_naive, df_masks_naive = nested_cross_validation(1,clf_class, param_grid, X_clean, y_clean, df_masks_knn)

# Save the results to CSV files
df_best_params_naive.to_csv(dir_results / "best_params_naive.csv", index=False)
df_masks_naive.to_csv(dir_results / 'masks_naive.csv', index=False)

for i in df_masks_naive:
    print(f"{i}: {df_masks_naive[i].values[0].sum()} features selected")

best_params_naive_ravel= pd.Series(df_best_params_naive.values.ravel())
best_params_naive_ravel.value_counts()[0:10]

Outer fold 1
Outer fold 2
Outer fold 3
Outer fold 4
Outer fold 5
Average performance across 1 trials: 0.7100 ± 0.0000
outer_1: 57 features selected
outer_2: 72 features selected
outer_3: 61 features selected
outer_4: 69 features selected
outer_5: 64 features selected


{'clf_class__var_smoothing': 0.1}      3
{'clf_class__var_smoothing': 1e-12}    2
Name: count, dtype: int64

# Code for the Random Forest Classifier
This code is used to perform nested cross-validation for hyperparameter tuning of a Random Forest classifier.
It includes feature selection using SelectKBest and RFECV, and it evaluates the model's performance using accuracy as the scoring metric.

In [90]:
# Imports

from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'clf_class__n_estimators': [50, 55, 60, 65, 70],
    'clf_class__max_depth': [3, 4, 5],
    'clf_class__max_features': [0.1, 0.2, 0.3, 1],
    # 'clf_class__min_samples_split': [7, 11, 15],
    # 'clf_class__min_samples_leaf': [1, 2, 3],
    # 'clf_class__class_weight': ['balanced', None],
    # 'clf_class__criterion': ['gini', 'entropy'],
}

clf_class = RandomForestClassifier()

# NUM_TRIALS = 1 -> could be changed when stronger computer available
nested_scores_rf, df_best_params_rf, df_masks_rf = nested_cross_validation(1,clf_class, param_grid, X_clean, y_clean, df_masks_knn)

# Save the results to CSV files
df_best_params_rf.to_csv(dir_results / "best_params_rf.csv", index=False)
df_masks_rf.to_csv(dir_results / 'masks_rf.csv', index=False)

for i in df_masks_rf:
    print(f"{i}: {df_masks_rf[i].values[0].sum()} features selected")

best_params_rf_ravel= pd.Series(df_best_params_rf.values.ravel())
best_params_rf_ravel.value_counts()[0:10]

Outer fold 1
Outer fold 2
Outer fold 3
Outer fold 4
Outer fold 5
Average performance across 1 trials: 0.7149 ± 0.0000
outer_1: 57 features selected
outer_2: 72 features selected
outer_3: 61 features selected
outer_4: 69 features selected
outer_5: 64 features selected


{'clf_class__max_depth': 3, 'clf_class__max_features': 1, 'clf_class__n_estimators': 70}      1
{'clf_class__max_depth': 4, 'clf_class__max_features': 1, 'clf_class__n_estimators': 55}      1
{'clf_class__max_depth': 5, 'clf_class__max_features': 0.2, 'clf_class__n_estimators': 65}    1
{'clf_class__max_depth': 5, 'clf_class__max_features': 0.3, 'clf_class__n_estimators': 55}    1
{'clf_class__max_depth': 4, 'clf_class__max_features': 0.2, 'clf_class__n_estimators': 65}    1
Name: count, dtype: int64

# Code for the Support Vector Machine classifier
This code is used to perform nested cross-validation for hyperparameter tuning of a Support Vector Machine (SVM) classifier.
It includes feature selection using SelectKBest and RFECV, and it evaluates the model's performance using accuracy as the scoring metric.

In [88]:
# Imports
from sklearn.svm import SVC

param_grid = {
    "clf_class__kernel": ["linear", "rbf", "poly"],
    "clf_class__C": [0.01, 0.05, 0.1, 1, 5, 10],
    "clf_class__gamma": ["scale", "auto"]
}

clf_class = SVC()

# NUM_TRIALS = 1 -> could be changed when stronger computer available
nested_scores_svm, df_best_params_svm, df_masks_svm = nested_cross_validation(1,clf_class, param_grid, X_clean, y_clean, df_masks_knn)

# Save the results to CSV files
df_best_params_svm.to_csv(dir_results / "best_params_svm.csv", index=False)
df_masks_svm.to_csv(dir_results / 'masks_svm.csv', index=False)

for i in df_masks_svm:
    print(f"{i}: {df_masks_svm[i].values[0].sum()} features selected")

best_params_svm_ravel= pd.Series(df_best_params_svm.values.ravel())
best_params_svm_ravel.value_counts()[0:10]

Outer fold 1
Outer fold 2
Outer fold 3
Outer fold 4
Outer fold 5
Average performance across 1 trials: 0.7693 ± 0.0000
outer_1: 57 features selected
outer_2: 72 features selected
outer_3: 61 features selected
outer_4: 69 features selected
outer_5: 64 features selected


{'clf_class__C': 0.1, 'clf_class__gamma': 'scale', 'clf_class__kernel': 'poly'}     2
{'clf_class__C': 10, 'clf_class__gamma': 'scale', 'clf_class__kernel': 'linear'}    1
{'clf_class__C': 10, 'clf_class__gamma': 'scale', 'clf_class__kernel': 'rbf'}       1
{'clf_class__C': 1, 'clf_class__gamma': 'scale', 'clf_class__kernel': 'linear'}     1
Name: count, dtype: int64


## Checken voor normaal distributie

In [ ]:
import pandas as pd
from scipy.stats import shapiro

# Keep only numeric features
numeric_df = df.select_dtypes(include='number')

# List to store names of normally distributed features
normally_distributed = []
not_normally_distributed = []
# Check normality using Shapiro-Wilk test
for col in numeric_df.columns:
    stat, p = shapiro(numeric_df[col])
    if p >= 0.05:
        normally_distributed.append(col)
    if p < 0.05:
        not_normally_distributed.append(col)

# Print the result
# print("Features with a normal distribution:")
# for feature in normally_distributed:
#    print(f"- {feature}")
# print("Features that are not normally distributed:")
# for feature in not_normally_distributed:
#    print(f"- {feature}")


Robust Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler

# Apply RobustScaler to the cleaned numeric columns
scaler = RobustScaler()

# Extract numeric columns again (post-cleaning & clipping)
numeric_df = df.select_dtypes(include='number')

# Fit and transform the data
scaled_data = scaler.fit_transform(numeric_df)

# Replace the original numeric columns in df with the scaled version
df[numeric_df.columns] = scaled_data

# Optional: print confirmation
print("Applied RobustScaler to numeric features.")


PCA Analysis and Visualization

This block performs Principal Component Analysis (PCA) to reduce dimensionality and visualize the 
structure of the dataset in 2D and 3D space. The data has been pre-cleaned and scaled.


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

# Define your class labels
y = df['label']  # 

# Step 1: Select numeric features
X = df.select_dtypes(include='number')

# Run full PCA
pca = PCA()
X_pca_full = pca.fit_transform(X)

# Calculate explained variance
explained_var = pca.explained_variance_ratio_
cumulative_var = np.cumsum(explained_var)

# Reduce to 80% variance
n_components = np.argmax(cumulative_var >= 0.80) + 1
print(f"Number of components to retain 80% variance: {n_components}")

# Project data to reduced PCA space
pca_reduced = PCA(n_components=n_components)
X_pca = pca_reduced.fit_transform(X)

# 2D PCA plot (PC1 vs PC2), color by class
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='Set1', edgecolor='k', alpha=0.7)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('2D PCA Projection (colored by class)')
plt.grid(True)
plt.legend(*scatter.legend_elements(), title="Class")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 2], c=y, cmap='Set1', edgecolor='k', alpha=0.7)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 3')
plt.title('2D PCA Projection (colored by class)')
plt.grid(True)
plt.legend(*scatter.legend_elements(), title="Class")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 1], X_pca[:, 2], c=y, cmap='Set1', edgecolor='k', alpha=0.7)
plt.xlabel('Principal Component 2')
plt.ylabel('Principal Component 3')
plt.title('2D PCA Projection (colored by class)')
plt.grid(True)
plt.legend(*scatter.legend_elements(), title="Class")
plt.tight_layout()
plt.show()

# 3D PCA plot (PC1 vs PC2 vs PC3), color by class
if n_components >= 3:
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], 
                    c=y, cmap='Set1', edgecolor='k', alpha=0.7)
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    ax.set_title('3D PCA Projection (colored by class)')
    
    # Add legend
    legend = ax.legend(*sc.legend_elements(), title="Class", loc="upper right")
    ax.add_artist(legend)
    
    plt.tight_layout()
    plt.show()
else:
    print("Not enough components for 3D plot.")
